In [1]:
!pip install h3
!pip install osmnx
!pip install osmium pyrosm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 26.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyrobuf-0.9.3-cp310-cp310-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.4 MB/s eta 0:00:00
  Created wheel for pyrosm: filename=pyrosm-0.6.2-cp310-cp310-linux_x86_64.whl size=2926366 sha256=200597af497686f7f4e86fb88d9d9ad7dabf7738510e4f12c59fda8634b02db2
  Stored in direc

In [20]:
import osmnx as ox
import pandas as pd
import numpy as np
import folium
import h3
from pyrosm import OSM
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import geopandas as gpd
from shapely.geometry import Point
from time import sleep

import warnings
warnings.filterwarnings("ignore")

In [17]:
# Определим функцию для выполнения запроса
def osm_query(tag, city):
    # Получаем объекты с нужным тегом в заданном городе
    gdf = ox.geometries_from_place(city, tag).reset_index()
    
    # Добавляем дополнительные столбцы для города, объекта и типа
    gdf['city'] = np.full(len(gdf), city.split(',')[0])
    gdf['object'] = np.full(len(gdf), list(tag.keys())[0])
    gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])

    # Добавляем информацию о названии и адресе (если они существуют)
    gdf['name'] = gdf.get('name', 'No name')  # если нет имени, ставим 'No name'
    gdf['address'] = gdf.get('address', 'No address')  # если нет адреса, ставим 'No address'
    
    # Оставляем только нужные столбцы
    gdf = gdf[['city', 'object', 'type', 'name', 'address', 'geometry']]
    
    print(f"Found {gdf.shape[0]} results for {tag} in {city}")
    return gdf

# Тег для поиска всех кафе
tag = {'amenity': 'cafe'}


# все что в голову взбрело
# tags = [
#         {'building' : 'apartments'}, {'building' : 'detached'}, 
#         {'building' : 'dormitory'}, {'building' : 'hotel'}, 
#         {'building' : 'house'}, {'building' : 'semidetached_house'}, 
#         {'building' : 'terrace'},  {'building' : 'commercial'},
#         {'building' : 'office'},  {'building' : 'terrace'},  
#         {'building' : 'terrace'}, {'building':'retail'}, 
#         {'building':'train_station'},
#         {'highway' : 'bus_stop'}, {'footway':'crossing'},
#         {'amenity':'cafe'}, {'amenity':'fast_food'}, 
#         {'amenity':'restaurant'}, {'amenity':'college'}, 
#         {'amenity':'language_school'},  {'amenity':'school'},  
#         {'amenity':'university'},  {'amenity':'atm'},  
#         {'amenity':'bank'},  {'amenity':'clinic'},  
#         {'amenity':'hospital'},  {'amenity':'pharmacy'},  
#         {'amenity':'theatre'},  {'amenity':'townhall'},  
#         {'amenity':'bench'}, 
#        ]

# торговые точки
# tags = [ 
#     {'shop': 'supermarket'},
#     {'shop': 'convenience'},
#     {'shop': 'bakery'},
#     {'shop': 'greengrocer'},
#     {'shop': 'butcher'},
#     {'shop': 'deli'},
#     {'shop': 'organic'},
#     {'shop': 'seafood'},
#     {'shop': 'alcohol'},
#     {'shop': 'beverages'},
#     {'shop': 'cheese'}
# ]

city = 'Москва, Россия'

# Выполним запрос и получим все кофейни в указанном городе
cafes_gdf = osm_query(tag, city)

#cafes_gdf = cafes_gdf.to_crs("EPSG:3857")
cafes_gdf['lat'] = cafes_gdf['geometry'].centroid.y
cafes_gdf['lon'] = cafes_gdf['geometry'].centroid.x

cafes_gdf.head()

/tmp/ipykernel_30/135577112.py:4: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index()


Found 4580 results for {'amenity': 'cafe'} in Москва, Россия


/tmp/ipykernel_30/135577112.py:66: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cafes_gdf['lat'] = cafes_gdf['geometry'].centroid.y
/tmp/ipykernel_30/135577112.py:67: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cafes_gdf['lon'] = cafes_gdf['geometry'].centroid.x


,city,object,type,name,address,geometry,lat,lon
0,Москва,amenity,cafe,Креветка,No address,POINT (37.52715 55.65170),55.651703,37.527154
1,Москва,amenity,cafe,Чайхана Ош,No address,POINT (37.68888 55.70522),55.705221,37.688884
2,Москва,amenity,cafe,Шоколадница,No address,POINT (37.67871 55.78971),55.789705,37.678715
3,Москва,amenity,cafe,Старый дуб,No address,POINT (37.78577 55.69769),55.697688,37.785774
4,Москва,amenity,cafe,Шоколадница,No address,POINT (37.57299 55.68667),55.686673,37.572991


In [ ]:
map_center = [55.7558, 37.6173]  # Координаты Москвы
mymap = folium.Map(location=map_center, zoom_start=12)

for idx, row in cafes_gdf.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Cafe: {row['type']}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(mymap)

mymap

In [45]:
# def get_street_by_coordinates(lat, lon):
#     # Инициализация геокодера Nominatim
#     geolocator = Nominatim(user_agent="Yarik")
    
#     # Выполняем обратное геокодирование для получения адреса
#     location = geolocator.reverse((lat, lon), language='ru', exactly_one=True)
#     sleep(0.2)
#     # Проверяем, есть ли результат
#     if location:
#         address = location.address
#         print(f"Адрес для координат ({lat}, {lon}): {address}")
#         return address
#     else:
#         print(f"Не удалось найти адрес для координат ({lat}, {lon})")
#         return None

# # Пример координат (Москва)
# latitude = 55.7558
# longitude = 37.6173

# address = get_street_by_coordinates(latitude, longitude)

Адрес для координат (55.7558, 37.6173): Московский Кремль и Красная Площадь, Дворцовая площадь, 18, 19, Тверской район, Москва, Центральный федеральный округ, 103073, Россия


In [23]:
osm = OSM("/kaggle/input/moskva-karta-low/Moscow.osm.pbf")
buildings = osm.get_buildings()
administrative_areas = osm.get_boundaries()

# Преобразуем в систему координат EPSG:4326
buildings = buildings.to_crs(epsg=4326)
administrative_areas = administrative_areas.to_crs(epsg=4326)

# Определение функции для извлечения информации по координатам
def get_address_info(longitude, latitude):
    point = Point(longitude, latitude)
    nearest_building_idx = buildings.geometry.distance(point).sort_values().index[0]
    building_info = buildings.loc[nearest_building_idx]
    
    # Извлекаем адрес, имя и площадь
    address = f"{building_info.get('addr:street', 'Неизвестно')}, {building_info.get('addr:housenumber', '')}"
    name = building_info.get('name', 'Имя не указано')
    area = building_info.geometry.area
    
    # Поиск района
    building_polygon = building_info.geometry
    district = administrative_areas[administrative_areas.contains(building_polygon)]
    district_name = district.iloc[0].get('name', 'Район не найден') if not district.empty else "Район не найден"
    
    return address, name, area, district_name

# Применяем функцию к DataFrame
cafes_gdf[['address', 'name', 'area', 'district']] = cafes_gdf.apply(
    lambda row: get_address_info(row['lon'], row['lat']),
    axis=1, result_type='expand'
)

In [25]:
cafes_gdf

,city,object,type,name,address,geometry,lat,lon,h3_indx,cafe_counts,area,district
0,Москва,amenity,cafe,None,"улица Бутлерова, 12",POINT (37.52715 55.65170),55.651703,37.527154,892cce8d24fffff,2,5.436523e-07,район Коньково
1,Москва,amenity,cafe,None,"Южнопортовая улица, 22 с10",POINT (37.68888 55.70522),55.705221,37.688884,892cce8e963ffff,1,1.309527e-07,Московская область
2,Москва,amenity,cafe,"Торговый центр ""Русское раздолье""","Сокольническая площадь, 4А",POINT (37.67871 55.78971),55.789705,37.678715,892cceb95c3ffff,3,2.245870e-07,Московская область
3,Москва,amenity,cafe,None,"улица Юных Ленинцев, 115",POINT (37.78577 55.69769),55.697688,37.785774,892cce84a97ffff,1,4.674075e-08,Московская область
4,Москва,amenity,cafe,None,"улица Дмитрия Ульянова, 24",POINT (37.57299 55.68667),55.686673,37.572991,892cce8de97ffff,3,1.141409e-06,Академический район
...,...,...,...,...,...,...,...,...,...,...,...,...
4575,Москва,amenity,cafe,None,"None, None","POLYGON ((37.62774 55.82573, 37.62774 55.82575...",55.825775,37.627914,892cce16c8bffff,9,1.388279e-07,Московская область
4576,Москва,amenity,cafe,None,"None, None","POLYGON ((37.62704 55.82600, 37.62705 55.82607...",55.826050,37.627157,892cce16c8bffff,9,1.388279e-07,Московская область
4577,Москва,amenity,cafe,None,"проспект Мира, 119 с516","POLYGON ((37.61979 55.82997, 37.61979 55.82998...",55.829953,37.619932,892cce16ccfffff,2,7.785275e-10,Московская область
4578,Москва,amenity,cafe,None,"Ярославская улица, 10 к2","POLYGON ((37.64787 55.82061, 37.64787 55.82065...",55.820645,37.647926,892cceb93d7ffff,2,2.029992e-09,Московская область
